# UC Subreddit Upvote Prediction
### Analyzing Post Features and Engagement Across the UC Sub-Reddit System


## 1. Introduction

Reddit is one of the largest discussion and bonding platforms for college communities, especially UC's. Each UC campus' subreddit has specific cultures, trends, and content patterns. These communities are especially useful for students to voice their opinions or share a funny story!

Take this as an example:

<img src="images\Screenshot 2025-11-12 225912.png" width="800">

This project aims to analyze Reddit posts from various UC subreddits to predict how many upvotes a post will earn based on its content, metadata, and engagement features. We thought this was interesting because you can analyze individual UC subreddits to see the differences in culture communities.

However, a practical use being if you were a club trying to have people go to your event:
What factors influence your post being brought up to the front page?
Or, how you could get the most engagement out of your post?

### **Practical Terminology**

Terms and Descriptions for Common Reddit Terms:
| Term | Meaning | Relevance to the Project |
|------|---------|---------------------------|
| **Upvote** | A + positive vote indicating support | Our main target variable |
| **Downvote** | A - negative vote indicating disagreement | Affects score, (negatively) but not always included |
| **Score** | Upvotes − Downvotes | Sometimes differs from "upvotes" field |
| **Upvote Ratio** | % of total votes that are upvotes | Proxy for sentiment/approval |
| **Karma** | A user’s total upvote score on Reddit | Used as a predictor of credibility |
| **OP** | "Original Poster": the person who created the post | We track OP karma & account age |
| **Flair** | A label for the post ("Funny", "School", etc.) | Helps to categorize content |
| **Mods** | Subreddit moderators | They influence which posts stay or get removed |
| **NSFW** | “Not Safe For Work” content flag | Included as a binary feature |
| **Hot** | A listing sorted by engagement + time decay | Affects which posts we scraped |
| **Top** | A listing sorted purely by score | May bias dataset toward high-engagement posts |
| **New** | A listing sorted by recent posts | Influences visibility + upvotes |
| **Shitpost** | Low-effort or joke post (Usually funny) | Often gets lots of votes in college subreddits |
| **Copypasta** | A repeated block of text/meme | Signals humor, may influence upvote behavior |


### **Research Questions**

**RQ1 (Prediction):**  
Can we predict the number of upvotes a post receives based on title features, metadata, and subreddit characteristics?

**RQ2 (Explanation):**  
Which factors such as title length, sentiment, posting time, media presence, and subreddit size have the strongest influence on upvote counts?

The goal is to combine exploratory analysis and machine learning to uncover meaningful patterns in campus-level Reddit engagement.

## 2. Study Design & Data Description

We scraped posts from the *top*, *hot*, and *new* listings for nine UC subreddits using Reddit’s public JSON API.
However, since these listings contain high-visibility posts, this dataset may be biased toward successful or recent posts.

### Subreddits included:
- r/UCSD  
- r/UCLA  
- r/Berkeley  
- r/UCSantaBarbara  
- r/UCI  
- r/UCDavis  
- r/UCSC  
- r/ucr  
- r/ucmerced  

### Dataset Structure
| Category | Variable | Description |
|----------|----------|-------------|
| **Post Content** | `title` | Text of the post title |
| | `title_length` | Length of the title (engineered feature) |
| | `sentiment` | Sentiment score of the title |
| **Engagement Metrics** | `upvotes` | Number of upvotes |
| | `upvote_ratio` | Proportion of upvotes / total votes |
| | `num_comments` | Number of comments|
| **Post Metadata** | `listing` | Where the post was scraped from: (top/hot/new) |
| | `created_utc` | Timestamp of post |
| | `hour` | Hour of day post was made (engineered feature) |
| **Content Type** | `has_media` | Whether the post contains media (0/1) |
| | `is_video` | Whether the post is a video (0/1) |
| | `over_18` | NSFW flag (0/1) |
| | `link_flair_text` | Category assigned by the subreddit |
| **Author Information** | `author` | Username of the creator |
| | `author_premium` | Whether the author is a Reddit Premium user |
| | `author_karma`| Total karma of the author |
| **Subreddit Information** | `subreddit` | Name of the UC subreddit |
| | `subreddit_subscribers` | Number of subscribers to that subreddit |


### Potential Biases:
Our dataset is subject to several sources of bias. First, scraping from the top listing leads to an overrepresentation of already high-performing posts, which may distort our understanding of typical engagement. Additionally, subreddit sizes vary widely—for example, UC Berkeley has far more traffic than UC Merced—so posts naturally receive different levels of exposure across campuses. The timing of data collection also affects upvote counts, since recently posted content has had less time to accumulate engagement. Finally, we lack historical data for older posts, which limits our ability to analyze long-term trends or normalize upvote counts over time. (NEED TO EDIT OARAGRAPH)

These variables allow both behavioral and structural analysis of community engagement.


In [10]:
import pandas as pd
import os

folder_path = "reddit data"

csv_files = [f for f in os.listdir(folder_path)]
csv_files

df_list = []

for file in csv_files:
    path = os.path.join(folder_path, file)
    temp = pd.read_csv(path)
    campus = file.replace(".csv", "")
    temp["campus"] = campus
    
    df_list.append(temp)

df_all = pd.concat(df_list, ignore_index=True)
df_all.head()

,subreddit,listing,title,author,upvotes,post_text,upvote_ratio,total_awards_received,score,edited,...,domain,link_flair_text,created_utc,subreddit_subscribers,author_premium,stickied,has_media,permalink,url,campus
0,Berkeley,top,DOJ announces investigation into UC Berkeley’s...,BayAreaNewsGroup,171,NaN,0.98,0,171,False,...,eastbaytimes.com,News,1.762894e+09,168611,False,False,True,https://www.reddit.com/r/berkeley/comments/1ou...,https://www.eastbaytimes.com/2025/11/10/uc-ber...,Berkeley
1,Berkeley,top,Why do you guys,Kitched_,70,Throw yourself at oncoming traffic? Imagine al...,0.89,0,70,False,...,self.berkeley,Local,1.762847e+09,168611,False,False,False,https://www.reddit.com/r/berkeley/comments/1ou...,https://www.reddit.com/r/berkeley/comments/1ou...,Berkeley
2,Berkeley,top,Plan to add housing in 3 wealthy Berkeley neig...,Generalaverage89,29,NaN,0.94,0,29,False,...,berkeleyside.org,News,1.762867e+09,168611,False,False,True,https://www.reddit.com/r/berkeley/comments/1ou...,https://www.berkeleyside.org/2025/11/07/berkel...,Berkeley
3,Berkeley,top,Anyone wanna be SnackPass friends?,happythepanda1,11,use snackpass relatively frequently but realiz...,0.83,0,11,False,...,self.berkeley,University,1.762917e+09,168611,False,False,False,https://www.reddit.com/r/berkeley/comments/1ou...,https://www.reddit.com/r/berkeley/comments/1ou...,Berkeley
4,Berkeley,top,Melt Banana is playing at Cornerstone on Novem...,joshhug,11,"Hello everyone, it's me, Josh Hug. I am not us...",1.00,0,11,1762929632.0,...,self.berkeley,Events/Organizations,1.762929e+09,168611,False,False,False,https://www.reddit.com/r/berkeley/comments/1ou...,https://www.reddit.com/r/berkeley/comments/1ou...,Berkeley


## 3. Exploratory Data Analysis (PT.1)

Before building any models, we first need to see the structure of the dataset and find any patterns in upvotes across UC subreddits. This helps us to see any early trends relevant to our research questions.

Some basic questions we had in mind are:

- How many posts did we collect across all UC campuses?
- Which subreddits have the most activity?
- How are upvotes distributed?
- Do different UC campuses show different upvote behaviors?

In [1]:
# We have about 4748 rows of data scraped from the combined subreddits
df_all.shape

NameError: name 'df_all' is not defined

In [17]:
df_all['campus'].value_counts()

campus
ucmerced          939
ucr               839
UCSC              736
UCDavis           634
UCI               531
UCSantaBarbara    428
UCSD              327
ucla              211
Berkeley          103
Name: count, dtype: int64